# Production mix - Model 10

## Situation
You own a boutique pottery business, making and selling two types of large ornamental products called Lunar Orb and Solar Disc. Given constraints on staff hours, available materials, and product sales, your objective is to maximize the total profit margin from the shop.

## Implementation
Linear Program (LP), using CVXPY. The data is loaded from an external json file.

Note: In Jupyter Lab a json file is opened, by default, in json format. To make changes to the file, open the file with the Editor (right-click > Open with > Editor).

## Source
Replicates an Excel model described in article "Production mix via graphical LP" at https://www.solvermax.com/blog/production-mix.

In [1]:
# Import dependencies

import cvxpy as cp
import pandas as pd
import numpy as np
import os.path
import json

In [2]:
# Get data

DataFilename = os.path.join('.', 'productiondata10.json')
with open(DataFilename, 'r') as f:
    Data = json.load(f)

In [3]:
# Declarations

Name = Data['Name']
Hours = Data['Hours']
kg = Data['kg']
SalesLimit = Data['SalesLimit']
VarInitial = Data['VarInitial']   # Not used
VarLBounds = Data['VarLBounds']
VarUBounds = Data['VarUBounds']
Engine = Data['Engine']
TimeLimit = Data['TimeLimit']

Coefficients = Data['Coefficients']
Products = list(Coefficients.keys())
NumProducts = len(Products)

Margin = np.zeros(NumProducts)
People = np.zeros(NumProducts)
Materials = np.zeros(NumProducts)
Sales = np.zeros(NumProducts)
for p in Products:
    i = int(p)
    Margin[i]    = Coefficients[p]['Margin']
    People[i]    = Coefficients[p]['People']
    Materials[i] = Coefficients[p]['Materials']
    Sales[i]     = Coefficients[p]['Sales']

In [4]:
# Define model

Production = cp.Variable(NumProducts)   # Variables

objective = cp.Maximize(cp.sum(Margin @ Production))   # Objectve function
constraints = []   # Constraints
constraints += [cp.sum(People @ Production) <= Hours]
constraints += [cp.sum(Materials @ Production) <= kg]
constraints += [cp.sum(Sales @ Production) <= SalesLimit]

constraints += [Production >= VarLBounds]   # Bounds on variables
constraints += [Production <= VarUBounds]

In [5]:
# Solve model

Model = cp.Problem(objective, constraints)

if Engine == 'cbc':
    EngineObj = cp.CBC
elif Engine == 'glop':
    EngineObj = cp.GLOP
elif Engine == 'glpk':
    EngineObj = cp.GLPK
elif Engine == 'cvxopt':
    EngineObj = cp.CVXOPT

Result = Model.solve(solver=EngineObj, verbose=True, max_seconds=TimeLimit)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Dec 13 11:02:27 AM: Your problem has 2 variables, 5 constraints, and 0 parameters.
(CVXPY) Dec 13 11:02:27 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 13 11:02:27 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 13 11:02:27 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 13 11:02:27 AM: Compiling problem (target solver=CVXOPT).
(CVXPY) Dec 13 11:02:27 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Con

In [6]:
# Process results

WriteSolution = False
Optimal = False
Condition = Model.status

if Condition == 'optimal':
    Optimal = True
    WriteSolution = True

In [7]:
# Write output

print(Name, '\n')
print('Status:', Model.status)
print('Solver:', Engine, '\n')

if WriteSolution:
    print(f"Total margin = ${objective.value:,.2f}\n")
    pd.options.display.float_format = "{:,.4f}".format
    ProductResults = pd.DataFrame()
    for p in Products:
        ProductResults.loc[p, 'Production'] = Production[int(p)].value
    display(ProductResults)
    
    ConstraintStatus = pd.DataFrame(columns=['Slack', 'Dual'])
    for c in range(3):
        ConstraintStatus.loc[c] = [constraints[c].expr.value, constraints[c].dual_value]
    display(ConstraintStatus)
else:
    print('No solution loaded\n')
    print('Model:')
    print(Model)

Boutique pottery shop - Model 10 

Status: optimal
Solver: cvxopt 

Total margin = $3,076.92



,Production
0,6.4103
1,12.8205


,Slack,Dual
0,-41.6667,0.0000
1,-0.0000,6.1538
2,0.0000,15.3846
